In [11]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import pickle
from ECI import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
with open('databytag.txt', 'rb') as f:
    data_by_tag0 = pickle.load(f)

In [3]:
data_by_tag = data_by_tag0.dropna()
data_by_tag.head()

,user,tag,tag_nbook,phone
0,1660379270,原创女频,59.0,3
1,1944430962,原创女频,52.0,3
2,2448925347,原创女频,49.0,3
3,1175473708,原创女频,57.0,3
4,2303800198,原创女频,62.0,2


In [4]:
userlist = list(data_by_tag['user'])
taglist = list(data_by_tag['tag'])
data = list(data_by_tag['tag_nbook'])

In [5]:
def get_dict(lst):
    return dict(zip(list(set(lst)),range(len(set(lst)))))

In [6]:
dx = get_dict(userlist)
dy = get_dict(taglist)
x = [dx[user] for user in userlist]
y = [dy[tag] for tag in taglist]

In [7]:
from scipy import sparse
nx,ny=len(dx),len(dy)
M = sparse.csc_matrix((data,(x,y)),shape=(nx,ny))

In [17]:
@timelogger
def RCA_sparse(M):
    B = M.sum(1)*M.sum(0) 
    Sum = M.sum()
    R = sparse.lil_matrix(M*Sum/B)
    R[R>=1]=1
    R[R<1]=0
    R = R.tocsc()
    return R

In [18]:
R = RCA_sparse(M)

/usr/local/lib/python3.5/dist-packages/scipy/sparse/compressed.py:287: SparseEfficiencyWarning: Comparing a sparse matrix with a scalar greater than zero using < is inefficient, try using >= instead.
  warn(bad_scalar_msg, SparseEfficiencyWarning)


==Time 1.52==


/usr/local/lib/python3.5/dist-packages/scipy/sparse/compressed.py:287: SparseEfficiencyWarning: Comparing a sparse matrix with a scalar greater than zero using < is inefficient, try using >= instead.
  warn(bad_scalar_msg, SparseEfficiencyWarning)


In [22]:
D = Get_eci_pci_sparse(R)

==Time 0.19==


In [29]:
res = [(key,D['pci'][dy[key]]) for key in dy]
sorted(res,key=lambda  x:x[1],reverse=True)

[('外文', 0.5939770247712045),
 ('经济管理', 0.5518702850851366),
 ('计算机', 0.5342391244652023),
 ('历史传记', 0.4702840018727088),
 ('科技', 0.4684287296504785),
 ('社会科学', 0.4333926062239216),
 ('成功励志', 0.42859409240065954),
 ('两性情感', 0.3853705531460898),
 ('文学艺术', 0.3240191322891341),
 ('亲子少儿', 0.32082208958506087),
 ('生活', 0.28857924732570733),
 ('期刊', 0.06278818087831857),
 ('小说', -0.15632418165751366),
 ('原创男频', -1.4300874000512351),
 ('原创女频', -3.275953485984874)]